In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

## Mouse aligned

In [ ]:
if not os.path.exists("build/thymus/ALIGNED_Mus_musculus_Thymus"):
    os.makedirs("build/thymus/ALIGNED_Mus_musculus_Thymus")
quake_10x_thymus = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_10x_Thymus/data.h5")
quake_10x_thymus.obs["cluster"] = "cluster_" + quake_10x_thymus.obs["cluster"].astype(int).astype(str)
quake_smart_seq2_thymus = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_Smart-seq2_Thymus/data.h5")
quake_smart_seq2_thymus.obs["cluster"] = "cluster_" + quake_smart_seq2_thymus.obs["cluster"].astype(int).astype(str)
mouse = cb.data.ExprDataSet.merge_datasets(dict(
    quake_10x_thymus=quake_10x_thymus,
    quake_smart_seq2_thymus=quake_smart_seq2_thymus
), merge_uns_slots=["seurat_genes"])

In [ ]:
mouse.obs.head()

In [ ]:
mouse.obs.dtypes

In [ ]:
opt_model_kwargs = dict(
    batch_effect=["dataset_name"], rmbatch_module_kwargs=dict(lambda_reg=0.05)
)
mouse_model = cb.directi.fit_DIRECTi(
    mouse, mouse.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
mouse.latent = mouse_model.inference(mouse)

In [ ]:
ax = mouse.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/thymus/ALIGNED_Mus_musculus_Thymus/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("dataset_name", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/thymus/ALIGNED_Mus_musculus_Thymus/dataset_name.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/thymus/ALIGNED_Mus_musculus_Thymus/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("gender", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/thymus/ALIGNED_Mus_musculus_Thymus/gender.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
mouse.write_dataset("build/thymus/ALIGNED_Mus_musculus_Thymus/ALIGNED_Mus_musculus_Thymus.h5")

In [ ]:
%%capture capio
mouse_models = [mouse_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    mouse_models.append(cb.directi.fit_DIRECTi(
        mouse, mouse.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
mouse_blast = cb.blast.BLAST(mouse_models, mouse)
mouse_blast.save("build/thymus/ALIGNED_Mus_musculus_Thymus")

In [ ]:
with open("build/thymus/ALIGNED_Mus_musculus_Thymus/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/thymus/ALIGNED_Mus_musculus_Thymus/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(mouse_blast, "build/thymus/ALIGNED_Mus_musculus_Thymus")

In [ ]:
%%writefile build/thymus/ALIGNED_Mus_musculus_Thymus/predictable.txt
cell_ontology_class